# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

<div class="alert alert-block alert-warning">
**Note:** Make sure you have selected a **Python 3** kernel in Workspaces or the hmm-tagger conda environment if you are running the Jupyter server on your own machine.
</div>

In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain,zip_longest
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. Review the reference below, then run and review the next few cells to make sure you understand the interface before moving on to the next step.

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [4]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


In [5]:
data.training_set.tagset

frozenset({'.',
           'ADJ',
           'ADP',
           'ADV',
           'CONJ',
           'DET',
           'NOUN',
           'NUM',
           'PRON',
           'PRT',
           'VERB',
           'X'})

<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [6]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [7]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [8]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
total = 0
for i, pair in enumerate(data.training_set.stream()):
    total += i
    #print(total)
    print("\t", pair)
    if i > 100: break


Stream (word, tag) pairs:

	 ('Whenever', 'ADV')
	 ('artists', 'NOUN')
	 (',', '.')
	 ('indeed', 'ADV')
	 (',', '.')
	 ('turned', 'VERB')
	 ('to', 'ADP')
	 ('actual', 'ADJ')
	 ('representations', 'NOUN')
	 ('or', 'CONJ')
	 ('molded', 'VERB')
	 ('three-dimensional', 'ADJ')
	 ('figures', 'NOUN')
	 (',', '.')
	 ('which', 'DET')
	 ('were', 'VERB')
	 ('rare', 'ADJ')
	 ('down', 'PRT')
	 ('to', 'ADP')
	 ('800', 'NUM')
	 ('B.C.', 'NOUN')
	 (',', '.')
	 ('they', 'PRON')
	 ('tended', 'VERB')
	 ('to', 'PRT')
	 ('reflect', 'VERB')
	 ('reality', 'NOUN')
	 ('(', '.')
	 ('see', 'VERB')
	 ('Plate', 'NOUN')
	 ('6a', 'NUM')
	 (',', '.')
	 ('9b', 'NUM')
	 (')', '.')
	 (';', '.')
	 (';', '.')
	 ('For', 'ADP')
	 ('almost', 'ADV')
	 ('two', 'NUM')
	 ('months', 'NOUN')
	 (',', '.')
	 ('the', 'DET')
	 ('defendant', 'NOUN')
	 ('and', 'CONJ')
	 ('the', 'DET')
	 ('world', 'NOUN')
	 ('heard', 'VERB')
	 ('from', 'ADP')
	 ('individuals', 'NOUN')
	 ('escaped', 'VERB')
	 ('from', 'ADP')
	 ('the', 'DET')
	 ('grave', 


For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

## Step 2: Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [9]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # TODO: Finish this function!
    unique_count_dict = defaultdict(lambda:defaultdict(int)) #returns a default dictionary of count 0 if no sequences of the word are found
    for tag,word in zip(sequences_A,sequences_B):
        unique_count_dict[tag][word] += 1
    return unique_count_dict


# Calculate C(t_i, w_i)
tag_seq = [tag for (word,tag) in data.training_set.stream()]
word_seq = [word for (word,tag) in data.training_set.stream()]
emission_counts = pair_counts(tag_seq,word_seq)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [10]:
word_counts1 = pair_counts(word_seq,tag_seq)
word_counts1 
for key,value in word_counts1.items():
    
    max_item = max(value.values())
    print(key,max_item)
    for pos,count in value.items():
        if count == max_item:
            print(pos)


Whenever 12
ADV
artists 34
NOUN
, 46499
.
indeed 92
ADV
turned 264
VERB
to 11784
PRT
actual 77
ADJ
representations 7
NOUN
or 3218
CONJ
molded 12
VERB
three-dimensional 8
ADJ
figures 69
NOUN
which 2844
DET
were 2596
VERB
rare 33
ADJ
down 543
PRT
800 10
NUM
B.C. 13
NOUN
they 2235
PRON
tended 18
VERB
reflect 21
VERB
reality 67
NOUN
( 1923
.
see 572
VERB
Plate 2
NOUN
6a 1
NUM
9b 1
NUM
) 1949
.
; 4418
.
For 522
ADP
almost 302
ADV
two 1051
NUM
months 142
NOUN
the 50287
DET
defendant 6
NOUN
and 22369
CONJ
world 548
NOUN
heard 189
VERB
from 3393
ADP
individuals 56
NOUN
escaped 15
VERB
grave 16
ADJ
about 972
ADP
fathers 9
NOUN
mothers 23
NOUN
graybeards 1
NOUN
adolescents 5
NOUN
babies 7
NOUN
starved 3
VERB
beaten 9
VERB
death 208
NOUN
strangled 5
VERB
machine-gunned 1
VERB
gassed 1
VERB
burned 31
VERB
. 39500
.
Clearer 1
ADJ
meaning 95
NOUN
Yes 69
ADV
gentlemen 18
NOUN
I 4140
PRON
am 176
VERB
getting 135
VERB
point 287
NOUN
my 951
DET
About 34
ADV
same 542
ADJ
time 1275
NOUN
Alleghenies 1
NOUN

Madeira 1
NOUN
Today's 6
NOUN
Voice 5
NOUN
incorrect 3
ADJ
Fromm's 14
NOUN
analysis 84
NOUN
alienation 20
NOUN
sphere 18
NOUN
production 116
NOUN
centers 37
NOUN
around 249
ADP
concepts 25
NOUN
bureaucratization 1
NOUN
corporation 52
NOUN
separation 13
NOUN
ownership 17
NOUN
broad 70
ADJ
view 129
NOUN
corporate 12
ADJ
ineffective 3
ADJ
dispersion 3
NOUN
entered 86
VERB
First 68
ADJ
World 88
NOUN
War 132
NOUN
Baker 25
NOUN
made 897
VERB
Draft 8
NOUN
1917 8
NUM
prohibited 5
VERB
sale 34
NOUN
liquor 30
NOUN
uniform 22
ADJ
provided 94
VERB
zones 2
NOUN
camps 17
NOUN
prostitution 7
NOUN
outlawed 3
VERB
reasons 79
NOUN
makes 132
VERB
superbly 8
ADV
useful 47
ADJ
psychiatric 3
ADJ
interview 26
NOUN
fact 372
NOUN
successfully 25
ADV
suggest 42
VERB
ways 97
NOUN
speed 54
NOUN
progress 87
NOUN
therapy 8
NOUN
bill 63
NOUN
passed 126
VERB
Assembly 22
NOUN
May 70
NOUN
pending 9
ADJ
careful 49
ADJ
scrutiny 12
NOUN
Are 50
VERB
partner 27
NOUN
connective 1
NOUN
ADJ
system 316
NOUN
network 27
NOUN
tail

hydroxyl-rich 2
ADJ
polyisocyanates 1
NOUN
tolylene 1
NOUN
diisocyanate 1
NOUN
popular 81
ADJ
remained 85
VERB
aside 45
ADV
Social 19
ADJ
widely 47
ADV
favored 16
VERB
gives 91
VERB
mind 232
NOUN
combinations 18
NOUN
old 452
ADJ
memories 13
NOUN
ideas 116
NOUN
experiences 40
NOUN
project 62
NOUN
environment 32
NOUN
ever-changing 4
ADJ
dance 57
NOUN
title 46
NOUN
nonstop 1
NOUN
format 3
NOUN
admirable 6
ADJ
continuity 21
NOUN
demands 36
NOUN
pauses 1
NOUN
identification 38
NOUN
dug 13
VERB
speech 57
NOUN
earlier 56
ADJ
Congressman 12
NOUN
decrying 1
VERB
statues 6
NOUN
monuments 6
NOUN
memorials 1
NOUN
dot 7
NOUN
landscape 16
NOUN
patriotic 9
ADJ
societies 28
NOUN
zealous 3
ADJ
constantly 31
ADV
hatching 2
VERB
NOUN
plans 80
NOUN
Bed 1
NOUN
slats 1
NOUN
washed 29
VERB
alum 1
NOUN
legs 53
NOUN
beds 8
NOUN
placed 98
VERB
cups 11
NOUN
kerosene 5
NOUN
woodwork 5
NOUN
liberally 4
ADV
corrosive 4
ADJ
sublimate 1
NOUN
feather 5
NOUN
charges 33
NOUN
electron 21
NOUN
proton 2
NOUN
believed 62
VE

NOUN
utility 23
NOUN
demanded 32
VERB
Lord 63
NOUN
confronted 25
VERB
loan 32
NOUN
Government 110
NOUN
India 43
NOUN
subsection 12
NOUN
G 11
NOUN
104 9
NUM
rupee 6
NOUN
equivalent 32
ADJ
$538 2
NOUN
financing 16
VERB
projects 47
NOUN
promote 26
VERB
balanced 19
VERB
mutually 9
ADV
agreed 65
VERB
return 74
NOUN
offered 67
VERB
U. 46
NOUN
S. 84
NOUN
Treasury 33
NOUN
bonds 33
NOUN
Warsaw 9
NOUN
gosh 1
PRT
want 255
VERB
promises 10
VERB
drinking 34
VERB
sickness 5
NOUN
Sat 1
VERB
sang 21
VERB
thinner 6
ADJ
5,500 1
NUM
private 148
ADJ
brand 10
NOUN
competition 44
NOUN
formulated 10
VERB
long-term 26
NOUN
researching 1
VERB
flux 21
NOUN
f 4
NOUN
nearest 14
ADJ
neighbors 31
NOUN
h 2
NOUN
e 2
NOUN
six 146
NUM
p 24
NOUN
remaining 34
VERB
intermediate 16
ADJ
numbers 91
NOUN
food 103
NOUN
Louchheim 1
NOUN
phase 53
NOUN
globetrotter 1
NOUN
knows 72
VERB
sections 54
NOUN
5 120
NUM
supplements 5
NOUN
hunch 4
NOUN
Willings 5
NOUN
ambush 5
NOUN
went 391
VERB
rules 57
NOUN
fairness 4
NOUN
Eyes 9
NOUN
h

Saturday 50
NOUN
orchestra 32
NOUN
sensibly 3
ADV
apparently 80
ADV
decent 14
ADJ
condition 71
NOUN
improved 42
VERB
November 61
NOUN
1887 2
NUM
connecting 5
VERB
several 269
ADJ
dwelling 6
NOUN
Dorset 3
NOUN
extended 46
VERB
kale 1
NOUN
hocking 1
VERB
respectable 15
ADJ
uptown 2
NOUN
returning 26
VERB
discretion 11
NOUN
moderation 3
NOUN
vague 17
ADJ
golden 22
ADJ
fading 4
VERB
rising 48
VERB
distortions 2
NOUN
window 103
NOUN
Third 11
ADJ
offer 54
VERB
adapted 11
VERB
country's 10
NOUN
spiritually 5
ADV
ran 109
VERB
Pete 19
NOUN
Vivian 9
NOUN
sympathy 26
NOUN
Gladden 3
NOUN
outspoken 6
ADJ
critic 21
NOUN
union's 2
NOUN
battle 62
NOUN
teamsters 6
NOUN
firemen 3
NOUN
1959 81
NUM
extracting 4
VERB
roleplaying 12
NOUN
previous 65
ADJ
twofold 2
ADJ
Middle 19
ADJ
Iraq 3
NOUN
Syria 2
NOUN
Egypt 13
NOUN
camp 47
NOUN
Lena 2
NOUN
Faber 4
NOUN
Hanover-Chalidale 1
NOUN
2:33 3
NUM
5.7 3
NUM
sets 33
NOUN
decline 17
NOUN
radios 5
NOUN
1961 98
NUM
nearly 103
ADV
15.0 1
NUM
- 51
ADP
15.5 1
NUM
paraps

...

Throw 1
NOUN
VERB
rebellions 1
NOUN
Targo 3
NOUN
wing-shooting 1
NOUN
Accident 1
NOUN
plain-clothesmen 2
NOUN
Sprague 2
NOUN
pesticides 1
NOUN
printable 1
ADJ
rootless 1
ADJ
girders 1
NOUN
pillows 1
NOUN
chambre's 1
X
Delaney 2
NOUN
O'Neill 2
NOUN
centimeter 2
NOUN
1-degree 1
NOUN
Assassination 1
NOUN
Mahler 1
NOUN
polishes 1
NOUN
critter 1
NOUN
Gouldings 1
NOUN
subscribing 1
VERB
Nan 1
NOUN
money-minded 1
ADJ
Floradora 1
NOUN
hansom 1
NOUN
two-run 1
ADJ
Athletics 2
NOUN
Writer 1
NOUN
flute 1
NOUN
cavort 1
VERB
K.C. 1
NOUN
cloth-of-gold 1
NOUN
155-yarder 1
NOUN
fenugreek 1
NOUN
cardamom 1
NOUN
deficits 1
NOUN
Lieutenant-Governor 1
NOUN
aspen 1
NOUN
panoramas 1
NOUN
Po 1
NOUN
hand-screened 1
ADJ
bookcases 1
NOUN
overdue 1
ADJ
alimony 1
NOUN
Collector 1
NOUN
exemptions 1
NOUN
stiffens 1
VERB
neighbours 1
NOUN
honeymooning 1
VERB
what's-his-name 1
NOUN
lounging 3
VERB
unalloyed 1
ADJ
bliss 2
NOUN
yokels 1
NOUN
dweller 2
NOUN
Wingman 1
NOUN
tablet 2
NOUN
toto 1
NOUN
Times-Picayune 1
NOUN
P

Stephen's 1
NOUN
Coconut 1
NOUN
conflagration 1
NOUN
forty-niners 1
NOUN
Handlers' 1
NOUN
Ass'ns' 1
NOUN
founder-originator 1
NOUN
latches 1
VERB
Rakestraw 1
NOUN
inclement 1
ADJ
Spa 1
NOUN
grizzled 1
ADJ
diplomat's 1
NOUN
Patriot 2
NOUN
T'ai-Shan 1
NOUN
Shantung 1
NOUN
mid-Victorian 1
ADJ
Flats 1
NOUN
Hockaday 1
NOUN
left-of-center 1
ADJ
mother-naked 1
ADJ
climaxed 1
VERB
unco-operative 1
ADJ
boyars 1
NOUN
irrigate 1
VERB
Baptiste 1
NOUN
Reinhardt 1
NOUN
Collins' 1
NOUN
Goodbody 2
NOUN
$15.5 1
NOUN
Consultation 1
NOUN
emasculated 1
VERB
nebula 1
NOUN
Divide 2
VERB
Longstreet 1
NOUN
subtler 1
ADJ
motion-picture 1
NOUN
ballyhooey 1
NOUN
presto 1
PRT
Valois 1
NOUN
Sicilian 3
ADJ
executor 1
NOUN
decedent 1
NOUN
leftfield 1
NOUN
Paschal 2
NOUN
Shartzer's 1
NOUN
combat-tested 1
ADJ
monsoon-shrouded 1
ADJ
road-shy 1
ADJ
guerrilla-th'-wisp 1
ADJ
savored 2
VERB
gangster 1
NOUN
Erwin 1
NOUN
Fife 1
NOUN
Marylanders 2
NOUN
enriching 1
VERB
gagged 1
VERB
pastor's 2
NOUN
caricature 1
VERB
NOUN
flow

Omega 1
NOUN
Sigma 1
NOUN
Grahamstown 1
NOUN
cost-billing 1
NOUN
24-sheet 1
ADJ
turnery 1
NOUN
hodgepodge 1
NOUN
cobwebs 1
NOUN
Taruffi 1
NOUN
two-lane 1
ADJ
165 2
NUM
algebraic 1
ADJ
riddles 2
NOUN
koan 1
X
Elks 1
NOUN
142 1
NUM
511 1
NUM
11-month-old 1
ADJ
10.6 1
NUM
Middle-Eastern 1
ADJ
tannin 1
NOUN
implausibly 1
ADV
hilltops 1
NOUN
caper 1
NOUN
maliciously 1
ADV
Lorena 1
NOUN
Gallon 1
NOUN
Gallon-Loren 1
NOUN
distresses 1
NOUN
shuns 1
VERB
unsuspecting 1
ADJ
needle-sharp 1
ADJ
fledglings 1
NOUN
astute 1
ADJ
authorizes 1
VERB
Hazards 1
NOUN
actuarially 1
ADV
mayhem 1
NOUN
Kromy 1
NOUN
sequel 1
NOUN
alias 1
NOUN
astronaut 1
NOUN
intrepid 1
ADJ
Varner 1
NOUN
Motion 1
NOUN
five-and-a-half 1
NUM
1907 1
NUM
1,107 1
NUM
conserving 1
VERB
Egalitarianism 1
NOUN
Popularism 1
NOUN
guests' 1
NOUN
dissenters 1
NOUN
fungus 1
NOUN
growths 1
NOUN
Derails 1
NOUN
efficacious 2
ADJ
Fought 1
VERB
Emcee 1
NOUN
Nixon's 1
NOUN
Anselmo 1
NOUN
Colzani 1
NOUN
hecatomb 1
NOUN
100-million-lb. 1
ADJ
shippers 

MacArthur 1
NOUN
32,589 1
NUM
pin-curl 1
NOUN
self-locking 1
ADJ
leathery 1
ADJ
topnotch 1
NOUN
cashed 1
VERB
convict's 1
NOUN
shrinks 1
VERB
Kelseyville 1
NOUN
Staunton 1
NOUN
$58,918 1
NOUN
$66,000 1
NOUN
$7,082 1
NOUN
judicious 1
ADJ
Registry 2
NOUN
impurity-doped 1
ADJ
Jenni 1
NOUN
DePaul 1
NOUN
Segovia's 1
NOUN
underwriting 1
VERB
swath 1
NOUN
asteroid 1
NOUN
remnants 1
NOUN
wounding 1
VERB
disbelieving 1
VERB
Burgundies 1
NOUN
Dissect 1
VERB
alternation 2
NOUN
Dronk's 1
NOUN
Baum 2
NOUN
untracked 1
ADJ
lords 1
NOUN
importunities 1
NOUN
55,987 1
NUM
Looked 1
VERB
Tractarians 1
NOUN
800,000 1
NUM
revels 1
NOUN
swanky 1
ADJ
writer-turned-painter 1
NOUN
Calm 1
ADJ
Henrik 1
NOUN
Kauffmann 1
NOUN
chlorothiazide 1
NOUN
edema 1
NOUN
endeavored 1
VERB
trade-preparatory 1
ADJ
Governors 2
NOUN
fallacy 1
NOUN
sponging 1
NOUN
corduroys 1
NOUN
Morphophonemic 1
ADJ
lowlands 1
NOUN
314 1
NUM
buckboard 1
NOUN
catapulting 1
VERB
recklessly 1
ADV
extremis 1
X
enforcers 1
NOUN
puberty 1
NOUN
0.25 1


NOUN
necklace 1
NOUN
6-foot-10 1
ADJ
brainwashing 1
NOUN
taut-nerved 1
ADJ
Constable's 1
NOUN
uninvited 1
ADJ
bonnet 1
NOUN
kerchief 1
NOUN
doc 1
NOUN
tip-toe 1
NOUN
Naturam 1
X
Pati 1
X
Senium 1
X
Idea 1
X
Platonica 1
X
sparkled 1
VERB
Quotations 1
NOUN
free-world 1
NOUN
marginally 1
ADV
epigrammatic 1
ADJ
utopianism 1
NOUN
protectively 1
ADV
elaboration 1
NOUN
creativity-oriented 1
ADJ
aspirant 2
NOUN
resigns 1
VERB
Westwood 1
NOUN
medicinal 1
ADJ
alkaloids 1
NOUN
Aroused 1
VERB
1846 1
NUM
Goodwin 1
NOUN
jeweler 1
NOUN
Equinox 1
NOUN
intraepithelial 1
ADJ
situ 1
X
ebb 1
VERB
offensively 1
ADV
Prompted 1
VERB
tenspot 1
NOUN
motioned 1
VERB
vacationland 1
NOUN
writing-like 1
ADJ
hasher 1
NOUN
hand-written 1
ADJ
Cate's 1
NOUN
squadron 1
NOUN
irrespective 1
ADJ
ivy 1
NOUN
green-scaled 1
ADJ
basileis 1
X
adulterated 1
VERB
peach 2
NOUN
plum 1
NOUN
life-size 1
NOUN
draper 1
NOUN
contiguous 1
ADJ
sambur 1
NOUN
elk 1
NOUN
astuteness 1
NOUN
nobles 1
NOUN
62-63 1
NUM
Redundant 1
ADJ
Archbishop

In [11]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

word_counts = pair_counts(word_seq,tag_seq)

mfc_table = {}
for key,value in word_counts.items():
    max_item = max(value.values())
    for pos,count in value.items():
        if count == max_item:
            mfc_table[key] = pos


# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

In [12]:
word_counts

defaultdict(<function __main__.pair_counts.<locals>.<lambda>()>,
            {'Whenever': defaultdict(int, {'ADV': 12}),
             'artists': defaultdict(int, {'NOUN': 34}),
             ',': defaultdict(int, {'.': 46499, 'X': 1}),
             'indeed': defaultdict(int, {'ADV': 92}),
             'turned': defaultdict(int, {'VERB': 264}),
             'to': defaultdict(int,
                         {'ADP': 8809,
                          'PRT': 11784,
                          'ADV': 1,
                          'NOUN': 1,
                          'X': 2}),
             'actual': defaultdict(int, {'ADJ': 77}),
             'representations': defaultdict(int, {'NOUN': 7}),
             'or': defaultdict(int, {'CONJ': 3218, 'X': 1}),
             'molded': defaultdict(int, {'VERB': 12}),
             'three-dimensional': defaultdict(int, {'ADJ': 8}),
             'figures': defaultdict(int, {'NOUN': 69, 'VERB': 2}),
             'which': defaultdict(int, {'DET': 2844, 'X': 1}),
    

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [13]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [14]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

In [15]:
data.testing_set.keys[:4]

['b100-28144', 'b100-23146', 'b100-35462', 'b100-37008']

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [16]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [17]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


## Step 3: Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmax}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [18]:
Counter(tag_seq)

Counter({'ADV': 44877,
         'NOUN': 220632,
         '.': 117757,
         'VERB': 146161,
         'ADP': 115808,
         'ADJ': 66754,
         'CONJ': 30537,
         'DET': 109671,
         'PRT': 23906,
         'NUM': 11878,
         'PRON': 39383,
         'X': 1094})

In [19]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO: Finish this function!
    return Counter(sequences)

# TODO: call unigram_counts with a list of tag sequences from the training set
tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
tag_unigrams = unigram_counts(tags)

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [20]:
data.training_set.Y[:3]
flat_list=[]
for tags in data.training_set.Y:
    ngrams = zip_longest(*[tags[i:] for i in range(2)])
    flat_list.append(ngrams)
bigram = []
for obj in flat_list:
    for tup in [*obj]:
        bigram.append(tup)
        
Counter(bigram)       

Counter({('ADV', 'NOUN'): 1478,
         ('NOUN', '.'): 62639,
         ('.', 'ADV'): 5124,
         ('ADV', '.'): 7577,
         ('.', 'VERB'): 9041,
         ('VERB', 'ADP'): 24927,
         ('ADP', 'ADJ'): 9533,
         ('ADJ', 'NOUN'): 43664,
         ('NOUN', 'CONJ'): 13185,
         ('CONJ', 'VERB'): 6012,
         ('VERB', 'ADJ'): 8423,
         ('.', 'DET'): 8008,
         ('DET', 'VERB'): 7062,
         ('ADJ', 'PRT'): 1301,
         ('PRT', 'ADP'): 2189,
         ('ADP', 'NUM'): 3467,
         ('NUM', 'NOUN'): 4524,
         ('.', 'PRON'): 5448,
         ('PRON', 'VERB'): 27860,
         ('VERB', 'PRT'): 9556,
         ('PRT', 'VERB'): 14886,
         ('VERB', 'NOUN'): 14230,
         ('NOUN', 'NUM'): 1783,
         ('NUM', '.'): 3210,
         ('.', 'NUM'): 1412,
         ('.', '.'): 12588,
         ('.', None): 44936,
         ('ADP', 'ADV'): 1805,
         ('ADV', 'NUM'): 597,
         ('DET', 'NOUN'): 68785,
         ('CONJ', 'DET'): 4636,
         ('NOUN', 'VERB'): 3497

In [21]:
tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
bigrams = [(tags[i], tags[i+1]) for i in range(len(tags) - 1)]
#Counter(bigrams)

In [22]:
flat_list1=[]
for word,tag in data.training_set.stream():
    ngrams = zip([tag[i:] for i in range(2)])
    flat_list1.append(ngrams)
flat_list1
big_list = [tup for obj in flat_list1 for tup in [*obj]]
#big_list

In [23]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """

    # TODO: Finish this function!
    bigrams = [(sequences[i], sequences[i+1]) for i in range(len(sequences) - 1)]
    return Counter(bigrams)

# TODO: call bigram_counts with a list of tag sequences from the training set
tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
bigrams = [(tags[i], tags[i+1]) for i in range(len(tags) - 1)]

    
tag_bigrams = bigram_counts(tags)
assert len(tags) == len(tag_seq)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [24]:
data.training_set.Y[:3]

(('ADV',
  'NOUN',
  '.',
  'ADV',
  '.',
  'VERB',
  'ADP',
  'ADJ',
  'NOUN',
  'CONJ',
  'VERB',
  'ADJ',
  'NOUN',
  '.',
  'DET',
  'VERB',
  'ADJ',
  'PRT',
  'ADP',
  'NUM',
  'NOUN',
  '.',
  'PRON',
  'VERB',
  'PRT',
  'VERB',
  'NOUN',
  '.',
  'VERB',
  'NOUN',
  'NUM',
  '.',
  'NUM',
  '.',
  '.',
  '.'),
 ('ADP',
  'ADV',
  'NUM',
  'NOUN',
  '.',
  'DET',
  'NOUN',
  'CONJ',
  'DET',
  'NOUN',
  'VERB',
  'ADP',
  'NOUN',
  'VERB',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  'CONJ',
  'NOUN',
  '.',
  'NOUN',
  '.',
  'NOUN',
  '.',
  'NOUN',
  '.',
  'VERB',
  '.',
  'VERB',
  'ADP',
  'NOUN',
  '.',
  'VERB',
  '.',
  'VERB',
  '.',
  'VERB',
  '.',
  'VERB',
  '.'),
 ('ADJ', 'NOUN'))

In [25]:
sequence = [seq[0] for seq in data.training_set.Y[:3]]
Counter(sequence)

Counter({'ADV': 1, 'ADP': 1, 'ADJ': 1})

In [26]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    # TODO: Finish this function!
    sequence = [seq[0] for seq in sequences]
    return Counter(sequence)

# TODO: Calculate the count of each tag starting a sequence
sequence = [seq[0] for seq in data.training_set.Y]
tag_starts = starting_counts(data.training_set.Y)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [27]:
sequence_end = [seq[-1] for seq in data.training_set.Y[:3]]
Counter(sequence_end)

Counter({'.': 2, 'NOUN': 1})

In [28]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    # TODO: Finish this function!
    return Counter(sequences)

# TODO: Calculate the count of each tag ending a sequence
sequence_end = [seq[-1] for seq in data.training_set.Y]
tag_ends = ending_counts(sequence_end)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [29]:

#tag_unigrams
#tag_bigrams
#tag_starts
#tag_ends

#tag_state_list = []
#for tag,word_count in emission_counts.items():
 #   total = float(sum(word_count.values()))
 #   distribution = {word: count/total for word, count in word_count.items()}
  #  tag_emissions = DiscreteDistribution(distribution)
  #  tag_state = State(tag_emissions, name=tag)
  #  basic_model.add_states(tag_state)
  #  tag_state_list.append(tag_state)
    
    


In [30]:
#for tag,word_dict in emission_counts.items():
#    distribution = {word: count for word, count in word_dict.items()}
    #print(distribution)

In [31]:
#start_prob = {}
#for tag,count in tag_starts.items():
#    start_prob[tag] = count/tag_unigrams[tag]
#start_prob
    

In [32]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)
tag_state_list = []
for tag,word_count in emission_counts.items():
    total = float(sum(word_count.values()))
    distribution = {word: count/total for word, count in word_count.items()}
    tag_emissions = DiscreteDistribution(distribution)
    tag_state = State(tag_emissions, name=tag)
    basic_model.add_states(tag_state)
    tag_state_list.append(tag_state)
    
#Adding start and end tags
#first we calculate the various probabilities for tags to be start tags then we add a custom start tag to create a markov sequence
start_prob = {}
for tag,count in tag_starts.items():
    start_prob[tag] = count/tag_unigrams[tag]

for tag_state in tag_state_list:
    basic_model.add_transition(basic_model.start, tag_state, start_prob[tag_state.name])

end_prob = {}

for tag,count in tag_ends.items():
    end_prob[tag] = count/tag_unigrams[tag]

for tag_state in tag_state_list:
    basic_model.add_transition(tag_state, basic_model.end, end_prob[tag_state.name])
    
# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)

# (Hint: you may need to loop & add transitions
transition_prob = {}
for tag1 in data.training_set.tagset:
    for tag2 in data.training_set.tagset:
        transition_prob[(tag1,tag2)] = tag_bigrams[(tag1, tag2)]/tag_unigrams[tag1]
        
for tag1_state in tag_state_list:
    for tag2_state in tag_state_list:
        basic_model.add_transition(tag1_state, tag2_state, transition_prob[(tag1_state.name, tag2_state.name)])



# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [33]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

training accuracy basic hmm model: 97.48%
testing accuracy basic hmm model: 95.86%


### Example Decoding Sequences with the HMM Tagger

In [34]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 


## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [35]:
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook HMM Tagger.ipynb to html',
 '[NbConvertApp] Writing 1290189 bytes to HMM Tagger.html',
 '[NbConvertApp] Converting notebook HMM Tagger-zh.ipynb to html',
 '[NbConvertApp] Writing 355982 bytes to HMM Tagger-zh.html',
 '[NbConvertApp] Converting notebook HMM warmup (optional).ipynb to html',
 '[NbConvertApp] Writing 334660 bytes to HMM warmup (optional).html',
 '[NbConvertApp] Converting notebook HMM warmup (optional)-zh.ipynb to html',
 '[NbConvertApp] Writing 321728 bytes to HMM warmup (optional)-zh.html']

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [36]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]